In [1]:
import pandas as pd
from business_models.databases import gdocs

from business_models.util.basic import send_mail

from business_models import gdocs_helper

In [4]:
import geopandas as gpd
import folium
from business_models import greenplum, hahn
from ast import literal_eval
import pandas as pd
from folium.features import GeoJson, GeoJsonTooltip, GeoJsonPopup
from shapely.geometry.geo import shape
from shapely.geometry import Point
import geojson

/usr/local/lib/python3.7/dist-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [5]:
def get_potential(orders_map):
    
    def get_new_lavka_orders(row):
        if row['eda_orders']>0: 
            return row['eda_orders'] * max(row['current_ratio'], row['base_close_ratio_orders'])
        else:
            return row['lavka_orders']
    
    orders_map['current_ratio'] = orders_map['lavka_orders']/orders_map['eda_orders']
    orders_map['new_lavka_orders'] = orders_map.apply(get_new_lavka_orders, axis=1)
    orders_map['lavka_orders_daily'] = orders_map['lavka_orders']/orders_map['days']
    orders_map['new_lavka_orders_daily'] = orders_map['new_lavka_orders']/orders_map['days']
    orders_map['diff'] = orders_map['new_lavka_orders'] - orders_map['lavka_orders']
    orders_map['diff_daily'] = orders_map['new_lavka_orders_daily'] - orders_map['lavka_orders_daily']
    
    return orders_map

def get_heatmap():
    
    def create_point(row):
        p = Point(row['lon'], row['lat'])
        return p

    orders_by_coord = greenplum(
        """
        Select * 
        From analyst.lavka_and_eda_orders_comparison
        """
    )
    
     
    orders_by_coord['geometry'] = orders_by_coord.apply(create_point, axis=1)
    orders_by_coord = get_potential(orders_by_coord)    
    
    return orders_by_coord

# Зоны

In [7]:
def get_splits(path_to_file):
    with open(path_to_file) as f:
        gj = gpd.read_file(f)
    return gj

In [8]:
def get_splits_from_text(txt):
    with open('tmp', 'w') as f:
        f.write(txt)
        
    with open('tmp') as f:
        gj = gpd.read_file(f)
    return gj

In [9]:
def get_current_lavka_zones():
    lavka_list = hahn("""
    SELECT external_id as place_id, location, cluster, title as lavka_name, ezones, status
    FROM hahn.`home/lavka/production/replica/postgres/wms/stores`;
    """)

    lavka_list = lavka_list[lavka_list['ezones']!='{}']

    lavka_list['shape'] = lavka_list['ezones'].apply(
        lambda a: geojson.Polygon(literal_eval(a)['main']['zone']['geometry']['coordinates'][0]))

    lavka_list = lavka_list[['place_id', 'cluster', 'lavka_name', 'shape']]

    lavka_list['place_id'] = lavka_list.place_id.astype(int)
    return lavka_list

# Pipeline

In [10]:
# Соберем статистику еды и лавки
heatmap = get_heatmap()
heatmap = gpd.GeoDataFrame(heatmap)

# Текущие границы лавок
lavka_list = get_current_lavka_zones()

/usr/local/lib/python3.7/dist-packages/business_models/databases/hahn.py:197: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  start = pd.datetime.now()


In [44]:
import shapely

In [11]:
# данный merge только ради названия лавок (нужно для унификации)
heatmap = heatmap.merge(lavka_list[['place_id', 'lavka_name']], on='place_id', how='left')

In [12]:
def get_actual_places(heatmap, splits, column_name):
    # оставим только актуальные для данного сплитования
    orders = gpd.sjoin(heatmap, splits, how='left')
    affected_lavkas = orders[~orders[column_name].isna()].place_id.unique()
    orders = orders[orders.place_id.isin(affected_lavkas)]
    return orders

# Делаем новые зоны

In [13]:
def new_zones_after_split(orders, column_name):
    lavka_to_split_dict = orders[['lavka_name', column_name]].dropna(
    ).drop_duplicates().groupby(['lavka_name']).agg(splits = (column_name ,  'unique')).to_dict()['splits']

    def get_new_geo(lavka_id, zones_list):
        new_geo = shape(lavka_list[lavka_list.lavka_name==lavka_id]['shape'].values[0])
        for zone in zones_list:
            new_geo = new_geo - splits[splits[column_name]==zone].geometry.values[0]
        return new_geo

    changed_lavka_area = [[i, get_new_geo(i, lavka_to_split_dict[i])] for i in lavka_to_split_dict]
    changed_lavka_area = gpd.GeoDataFrame(changed_lavka_area, columns=['lavka_name', 'geometry'])
    geometry_after_split = pd.concat([changed_lavka_area, splits[[column_name,'geometry']]])
    geometry_after_split.loc[geometry_after_split.lavka_name.isna(
    ),'lavka_name'] = geometry_after_split.loc[geometry_after_split.lavka_name.isna(),column_name]
    geometry_after_split = geometry_after_split[['lavka_name', 'geometry']].set_index('lavka_name')
    return geometry_after_split

# Calc Effect

In [14]:
def get_stat_table(df, geo, column_name):
    current_state = df.groupby('lavka_name')[['lavka_orders_daily']].sum()
    current_state['lavka_orders_daily'] = current_state['lavka_orders_daily'].round(1)
    
    df.loc[~df[column_name].isna(), 'lavka_name'] = df[column_name]
    result_table = df.groupby('lavka_name'
                             ).agg({'lavka_orders_daily':'sum', 
                                    'new_lavka_orders_daily':'sum', 
                                    'diff_daily':'sum',
                                    'first_day':'max'})
    
    result_table = result_table.join(current_state, how='left', rsuffix='_before_split').fillna(0)

    result_table['lavka_orders_daily'] = result_table['lavka_orders_daily'].round(1)
    result_table['new_lavka_orders_daily'] = result_table['new_lavka_orders_daily'].round(1)
    result_table['diff_daily'] = result_table['diff_daily'].round(1)

    
    result_table['diff_conf'] = result_table['diff_daily'] * (
        1 * result_table['lavka_orders_daily_before_split']==0.0)
    result_table['diff_non_conf'] = result_table['diff_daily'] - result_table['diff_conf']
    result_table['first_day'] = result_table['first_day'].apply(lambda x: str(x))
    result_table = geo.join(result_table)
    
    return result_table

In [18]:
def get_name_field(s):
    if 'description' in s.columns:
        return 'description'
    if 'name' in s.columns:
        return 'name'
    if 'address' in s.columns:
        return 'address'
    else:
        return 'something'


def main(heatmap, splits):
    column_name = get_name_field(splits)
    
    if column_name == 'something':
        splits['description'] = 'Сплит'
        column_name = 'description'
        
    splits  = splits.fillna({'description':'Сплит'})
    orders = get_actual_places(heatmap, splits, column_name)
    new_geometry = new_zones_after_split(orders, column_name)
    result_table = get_stat_table(orders, new_geometry, column_name)
    result_table_for_geo = result_table.reset_index()
    result_table_for_geo['old_zone'] = 1*(result_table_for_geo['lavka_orders_daily_before_split']>0)
    result_table = result_table[['lavka_orders_daily_before_split',
                                 'lavka_orders_daily', 
                                 'new_lavka_orders_daily', 
                                 'diff_daily', 'diff_conf', 
                                 'diff_non_conf', 
                                 'first_day']]

    result_table.loc['Total',:]= result_table.sum(axis=0)
    result_table.loc['Total','first_day']= None
    

    result_table = result_table.rename(columns={
        'lavka_orders_daily_before_split':'Было заказов до сплитования',
        'lavka_orders_daily':'Заказов в зоне сейчас', 
        'new_lavka_orders_daily':'Потенциал заказов в зоне', 
        'diff_daily':'Полный прирост',
        'diff_conf':'Прирост в сплите',
        'diff_non_conf':'Прирост оставшихся зон',
        'first_day':'Первый день продаж'})
    return result_table, result_table_for_geo

In [19]:
def create_map(result_table_for_geo):
    m = folium.Map(location=[55.777157, 37.436088], zoom_start=11)

    choropleth = folium.Choropleth(
        geo_data=result_table_for_geo.to_json(),
        name="choropleth",
        data=result_table_for_geo,
        columns=["lavka_name", "old_zone"],
        key_on="feature.properties.lavka_name",
#         fill_color="YlGn",
        fill_opacity=0.7,
        line_opacity=0.2,
        show=True

    )


    
    # Display Region Label
    choropleth.geojson.add_child( folium.features.GeoJsonTooltip(
        fields=['lavka_name', 'lavka_orders_daily', 'new_lavka_orders_daily', 'diff_conf', 'diff_non_conf'], 
        aliases=['Название', 'Заказов сейчас', 'Заказов будет', 'Прирост в сплите', 'Прирост оставшихся зон']
    )
    )

    
    for key in choropleth._children:
        if key.startswith('color_map'):
            del(choropleth._children[key])

    choropleth.add_to(m)

#     folium.LayerControl().add_to(m)


    return m

# Основной цикл

In [20]:
import datetime

In [ ]:
zones_for_split = gdocs_helper.read_table('1kOnZhDEHDPrhJaZSZxNaV8YnRf0HigeN9aVj6tqXs6E', 'Список', )
to_process = zones_for_split[zones_for_split['Processed']!='1']

for q in to_process.iterrows():
    split_name = q[1]['Name']
    coord = q[1]['Coord']
    splits = get_splits_from_text(coord)
    splits = splits[splits.geometry.apply(lambda x: x.type=='Polygon')]
    zones_for_split.loc[zones_for_split.Name==split_name, 'Processed'] = '1'
    zones_for_split.loc[zones_for_split.Name==split_name, 'Date'] = str(datetime.datetime.today())

    res, result_table_for_geo = main(heatmap, splits)
    map_of_split = create_map(result_table_for_geo)
    map_of_split.save(f'./{split_name}.html')
    map_of_split.save(f'./map.html')
    mail = q[1]['email']
    
    if (mail!=''):
        send_mail('aaraskin', mail, f'Сплитование {split_name}', 
                  res.to_html(), files=[f'./map.html'], use_suffix=True)

    gdocs_helper.upload_table('1kOnZhDEHDPrhJaZSZxNaV8YnRf0HigeN9aVj6tqXs6E', split_name, res.reset_index())
    
    graphic_orders = graphic_norm * res.loc['Total','Прирост в сплите']
    forecast = pd.DataFrame(graphic_orders).round(1)
    forecast = forecast.rename(columns={'count':'Прирост в сплите'})

    blank = pd.DataFrame([['-', '-']])
    blank = blank.rename(columns={0:' ', 1:'  '})
    
    gdocs.write(blank, split_name, '1kOnZhDEHDPrhJaZSZxNaV8YnRf0HigeN9aVj6tqXs6E', if_exists='append')
    gdocs.write(forecast, 
                split_name, 
                '1kOnZhDEHDPrhJaZSZxNaV8YnRf0HigeN9aVj6tqXs6E', 
                if_exists='append',
                index=True)
    
    gdocs_helper.upload_table('1kOnZhDEHDPrhJaZSZxNaV8YnRf0HigeN9aVj6tqXs6E', 
                              'Список', 
                              zones_for_split, 
                              with_drop=True)
    
    

# 

In [24]:
dyn = greenplum("""
Select place_id, date_trunc('month', msk_created_dttm::date), count(*)
from eda_cdm_marketplace.dm_order
where region_name='Москва'
and brand_store_flg
and confirmed_flg
and place_id in (
    Select place_id from eda_cdm_marketplace.dm_order
    where msk_created_dttm::date<'2020-01-01'::date
    )
group by place_id, date_trunc('month', msk_created_dttm::date)

""")

In [25]:
dyn = dyn.sort_values(by=['place_id', 'date_trunc'])

In [26]:
q = pd.merge(dyn.groupby('place_id', as_index=False)[['date_trunc']].min(), dyn, on='place_id')

In [27]:
def get_month_num(row):
    return (row.date_trunc_y.month - row.date_trunc_x.month) + (row.date_trunc_y.year - row.date_trunc_x.year) * 12

In [28]:
q['month'] = q.apply(get_month_num, axis=1)

In [29]:
graphic = q[q.month.between(5, 15)]
graphic['month'] = graphic['month']-3
graphic = graphic.groupby('month')['count'].sum()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [31]:
graphic_norm = graphic/graphic.min()

In [32]:
graphic_norm

month
2     1.000000
3     1.034044
4     1.021539
5     1.015016
6     1.040366
7     1.094437
8     1.118598
9     1.162978
10    1.158825
11    1.156323
12    1.248591
Name: count, dtype: float64